In [1]:
import tensorflow as tf
import tensorflow3d as t3d
import numpy as np

In [2]:
pt_sample=np.random.rand(1, 2048, 3).astype('float32')
flownet3d = t3d.models.FlowNet3D(name='flownet3d').build(input_shape1=(2048, 3), input_shape2=(2048, 3))
flownet3d.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 2048, 3)]    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 2048, 3)]    0                                            
__________________________________________________________________________________________________
set_conv (SetConv)              ((None, 1024, 3), (N 3808        input_1[0][0]                    
__________________________________________________________________________________________________
set_conv_2 (SetConv)            ((None, 1024, 3), (N 3808        input_2[0][0]                    
_______________________________________________________________________________________

In [3]:
flownet3d((pt_sample, pt_sample))

<tf.Tensor: shape=(1, 2048, 3), dtype=float32, numpy=
array([[[ 0.02515893, -0.12718855,  0.05282919],
        [-0.00070373, -0.0036317 ,  0.00125613],
        [-0.00108259, -0.00339303,  0.00212296],
        ...,
        [-0.0005703 , -0.00388454,  0.00138087],
        [-0.00105531, -0.00313573,  0.00184298],
        [-0.00121647, -0.00306972,  0.00211706]]], dtype=float32)>

In [4]:
pred = flownet3d((pt_sample, pt_sample))